# Gated Recurrent Unit (GRU) for SMS Spam Classification

This notebook trains and evaluates a GRU model with bidirectional layers for SMS spam detection

## GRU Model for SMS Spam Detection
 **Student:** Edith Githinji
 **Model:** GRU (Gated Recurrent Unit)
 **Embeddings Tested:** TF-IDF, Skip-gram, CBOW

In [ ]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import os


sys.path.append('..')

# Machine learning imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer

# TensorFlow imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(f"TensorFlow version: {tf.__version__}")
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")

### 2. Load and Explore Data

In [5]:
# Load the dataset
data_path = "../data/sms+spam+collection/SMSSpamCollection"
try:
    df = pd.read_csv(data_path, sep='\t', names=['label', 'message'])
    print(f" Successfully loaded {len(df)} messages")
except Exception as e:
    print(f"❌ Error loading data: {e}")
    # Try alternative path
    data_path = "data/sms+spam+collection/SMSSpamCollection"
    df = pd.read_csv(data_path, sep='\t', names=['label', 'message'])

 Successfully loaded 5572 messages


In [6]:
# Display basic info
print("\n Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Check class distribution
print("\n Class Distribution:")
print(df['label'].value_counts())
print(f"\nSpam percentage: {(df['label'] == 'spam').mean():.2%}")

# Show samples
print("\n Sample Messages:")
print(df.head())


 Dataset Info:
Shape: (5572, 2)
Columns: ['label', 'message']

 Class Distribution:
label
ham     4825
spam     747
Name: count, dtype: int64

Spam percentage: 13.41%

 Sample Messages:
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...



## 3. Preprocess Data


In [7]:
# Simple text cleaning function
def clean_text(text):
    """Basic text cleaning"""
    import re
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# Apply cleaning
df['clean_message'] = df['message'].apply(clean_text)

# Convert labels to binary (0=ham, 1=spam)
df['label_binary'] = df['label'].apply(lambda x: 1 if x == 'spam' else 0)

# Check cleaned data
print("Cleaned sample:")
print(df[['message', 'clean_message', 'label', 'label_binary']].head())

Cleaned sample:
                                             message  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   

                                       clean_message label  label_binary  
0  go until jurong point crazy available only in ...   ham             0  
1                            ok lar joking wif u oni   ham             0  
2  free entry in a wkly comp to win fa cup final ...  spam             1  
3        u dun say so early hor u c already then say   ham             0  
4  nah i dont think he goes to usf he lives aroun...   ham             0  


## 4. Split Data

In [ ]:
# Paths to processed split files
train_path = "../data/processed/train.tsv"
val_path   = "../data/processed/val.tsv"
test_path  = "../data/processed/test.tsv"

# Load the splits
train_df = pd.read_csv(train_path, sep='\t', names=['label', 'message'])
val_df   = pd.read_csv(val_path, sep='\t', names=['label', 'message'])
test_df  = pd.read_csv(test_path, sep='\t', names=['label', 'message'])

# Clean the messages
for df_split in [train_df, val_df, test_df]:
    df_split['clean_message'] = df_split['message'].apply(clean_text)
    df_split['label_binary'] = df_split['label'].apply(lambda x: 1 if x == 'spam' else 0)

# Extract features and labels
X_train, y_train = train_df['clean_message'].values, train_df['label_binary'].values
X_val, y_val     = val_df['clean_message'].values, val_df['label_binary'].values
X_test, y_test   = test_df['clean_message'].values, test_df['label_binary'].values

# Print data info
print(f"Train samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Test samples: {len(X_test)}")

# Check class balance in train
print(f"\nClass balance in Train: {pd.Series(y_train).value_counts().to_dict()}")


 Data Split:
Train: 3342 samples (60.0%)
Validation: 1115 samples (20.0%)
Test: 1115 samples (20.0%)

 Class Balance in Train: {0: 2894, 1: 448}


## 5. GRU Model Implementation

In [9]:
def create_gru_model(vocab_size, embedding_dim=100, hidden_dim=128, 
                    num_layers=2, dropout_rate=0.5, max_len=100,
                    embedding_matrix=None):
    """
    Create a GRU-based spam classifier
    """
    inputs = Input(shape=(max_len,))
    
    # Embedding layer
    if embedding_matrix is not None:
        embedding_layer = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_matrix.shape[1],
            weights=[embedding_matrix],
            input_length=max_len,
            trainable=False  # Can be True to fine-tune
        )(inputs)
    else:
        embedding_layer = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            input_length=max_len
        )(inputs)
    
    # Add dropout to embeddings
    x = Dropout(dropout_rate)(embedding_layer)
    
    # Stack multiple GRU layers
    for i in range(num_layers):
        return_sequences = (i < num_layers - 1)  # Return sequences for all but last layer
        gru_layer = Bidirectional(GRU(hidden_dim, return_sequences=return_sequences, dropout=dropout_rate))
        x = gru_layer(x)
        if return_sequences:
            x = Dropout(dropout_rate)(x)
    
    # Classifier
    x = Dense(64, activation='relu')(x)
    x = Dropout(dropout_rate/2)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

## 6. Helper Functions

In [10]:
def prepare_sequences(texts, tokenizer, max_len=100):
    """Convert texts to padded sequences"""
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')
    return padded

def build_tokenizer(texts, max_words=10000):
    """Build tokenizer from texts"""
    tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
    tokenizer.fit_on_texts(texts)
    
    vocab_size = min(max_words, len(tokenizer.word_index) + 1)
    print(f"Vocabulary size: {vocab_size}")
    print(f"Most common words: {list(tokenizer.word_index.keys())[:10]}")
    
    return tokenizer, vocab_size

def evaluate_model(model, X, y):
    """Evaluate model and return metrics"""
    predictions = (model.predict(X, verbose=0) > 0.5).astype(int)
    
    metrics = {
        'accuracy': accuracy_score(y, predictions),
        'f1': f1_score(y, predictions),
        'precision': precision_score(y, predictions),
        'recall': recall_score(y, predictions)
    }
    
    return metrics, predictions

## EXPERIMENT 1: GRU with TF-IDF Embeddings

In [11]:
print("=" * 60)
print("EXPERIMENT 1: GRU with TF-IDF Embeddings")
print("=" * 60)

EXPERIMENT 1: GRU with TF-IDF Embeddings


### 7.1 Prepare TF-IDF Features

In [12]:
# Build tokenizer
tokenizer_tfidf, vocab_size_tfidf = build_tokenizer(X_train)

# Prepare sequences
max_len = 100
X_train_seq = prepare_sequences(X_train, tokenizer_tfidf, max_len)
X_val_seq = prepare_sequences(X_val, tokenizer_tfidf, max_len)
X_test_seq = prepare_sequences(X_test, tokenizer_tfidf, max_len)

print(f"Sequence shape - Train: {X_train_seq.shape}")

Vocabulary size: 6417
Most common words: ['<OOV>', 'to', 'i', 'you', 'a', 'the', 'u', 'and', 'in', 'is']
Sequence shape - Train: (3342, 100)


### 7.2 Train GRU with TF-IDF Context

In [ ]:
vocab_size = vocab_size_tfidf
embedding_dim = 100
hidden_dim = 128
num_layers = 2
dropout_rate = 0.5

# Create model
model_tfidf = create_gru_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout_rate=dropout_rate,
    max_len=max_len,
    embedding_matrix=None
)

# Compile model
model_tfidf.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2),
    ModelCheckpoint('best_model_tfidf.h5', monitor='val_f1', 
                    save_best_only=True, mode='max')
]

print("\n Training GRU with TF-IDF context...")
print(model_tfidf.summary())

# Train model
history_tfidf = model_tfidf.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=15,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)


🚀 Training GRU with TF-IDF context...


/Users/sola/.local/share/virtualenvs/text-classification-groupwork-ObVFzkD4/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 100)       │       641,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 256)       │       176,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 256)            │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,131,301 (4.32 MB)

...

53/53 ━━━━━━━━━━━━━━━━━━━━ 15s 281ms/step - accuracy: 0.9892 - loss: 0.0380 - val_accuracy: 0.9686 - val_loss: 0.1120 - learning_rate: 0.0010
Epoch 4/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step - accuracy: 0.9921 - loss: 0.0286

53/53 ━━━━━━━━━━━━━━━━━━━━ 15s 291ms/step - accuracy: 0.9928 - loss: 0.0274 - val_accuracy: 0.9785 - val_loss: 0.0979 - learning_rate: 0.0010
Epoch 5/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 0.9964 - loss: 0.0175

53/53 ━━━━━━━━━━━━━━━━━━━━ 15s 290ms/step - accuracy: 0.9949 - loss: 0.0196 - val_accuracy: 0.9785 - val_loss: 0.0930 - learning_rate: 5.0000e-04
Epoch 6/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 0.9977 - loss: 0.0121

53/53 ━━━━━━━━━━━━━━━━━━━━ 16s 293ms/step - accuracy: 0.9970 - loss: 0.0122 - val_accuracy: 0.9821 - val_loss: 0.0885 - learning_rate: 5.0000e-04
Epoch 7/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 0.9984 - loss: 0.0081

53/53 ━━━━━━━━━━━━━━━━━━━━ 15s 292ms/step - accuracy: 0.9979 - loss: 0.0084 - val_accuracy: 0.9776 - val_loss: 0.0999 - learning_rate: 2.5000e-04


## 7.3 Evaluate TF-IDF Model

In [55]:
test_metrics_tfidf, preds_tfidf = evaluate_model(model_tfidf, X_test_seq, y_test)

print("\n TF-IDF Model - Test Results:")
print(f"Accuracy:  {test_metrics_tfidf['accuracy']:.4f}")
print(f"F1 Score:  {test_metrics_tfidf['f1']:.4f}")
print(f"Precision: {test_metrics_tfidf['precision']:.4f}")
print(f"Recall:    {test_metrics_tfidf['recall']:.4f}")

# Save results
results_tfidf = {
    'embedding': 'TF-IDF',
    'test_metrics': test_metrics_tfidf,
    'train_history': history_tfidf.history
}

# Create confusion matrix
cm_tfidf = confusion_matrix(y_test, preds_tfidf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_tfidf, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
plt.title('Confusion Matrix - GRU with TF-IDF')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('../results/figures/gru_tfidf_confusion.png', dpi=300, bbox_inches='tight')
plt.show()


 TF-IDF Model - Test Results:
Accuracy:  0.9794
F1 Score:  0.9193
Precision: 0.9632
Recall:    0.8792


ModuleNotFoundError: No module named 'matplotlib_inline'

## EXPERIMENT 2: GRU with Skip-gram Embeddings

In [57]:
print("\n" + "=" * 60)
print("EXPERIMENT 2: GRU with Skip-gram Embeddings")
print("=" * 60)


EXPERIMENT 2: GRU with Skip-gram Embeddings


In [60]:
from gensim.models import Word2Vec

print("=" * 60)
print("EXPERIMENT 2: GRU with Skip-gram Embeddings - Training Skip-gram")
print("=" * 60)

# Prepare sentences
sentences = [text.split() for text in all_texts]

# Train Word2Vec skip-gram model
skipgram_model = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=2,
    sg=1,  # sg=1 for skip-gram
    workers=4,
    seed=SEED
)

# Create embedding matrix
embedding_dim = 100
embedding_matrix_skipgram = np.zeros((vocab_size, embedding_dim))

word_index = tokenizer_tfidf.word_index
for word, idx in word_index.items():
    if idx < vocab_size:
        if word in skipgram_model.wv:
            embedding_matrix_skipgram[idx] = skipgram_model.wv[word]
        else:
            # Initialize with small random values
            embedding_matrix_skipgram[idx] = np.random.normal(scale=0.1, size=(embedding_dim,))

print(f"✓ Skip-gram embedding matrix shape: {embedding_matrix_skipgram.shape}") 


print("=" * 60)
print("EXPERIMENT 2: GRU with Skip-gram Embeddings - Training Skip-gram")
print("=" * 60)

# Prepare sentences
sentences = [text.split() for text in all_texts]

# Train Word2Vec skip-gram model
skipgram_model = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=2,
    sg=1,  # sg=1 for skip-gram
    workers=4,
    seed=SEED
)

# Create embedding matrix
embedding_dim = 100
embedding_matrix_skipgram = np.zeros((vocab_size, embedding_dim))

word_index = tokenizer_tfidf.word_index
for word, idx in word_index.items():
    if idx < vocab_size:
        if word in skipgram_model.wv:
            embedding_matrix_skipgram[idx] = skipgram_model.wv[word]
        else:
            # Initialize with small random values
            embedding_matrix_skipgram[idx] = np.random.normal(scale=0.1, size=(embedding_dim,))

print(f"✓ Skip-gram embedding matrix shape: {embedding_matrix_skipgram.shape}")

EXPERIMENT 2: GRU with Skip-gram Embeddings - Training Skip-gram
✓ Skip-gram embedding matrix shape: (6417, 100)
EXPERIMENT 2: GRU with Skip-gram Embeddings - Training Skip-gram


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


✓ Skip-gram embedding matrix shape: (6417, 100)


### 8.2 Train GRU with Skip-gram Embeddings

In [61]:
# Create model with skip-gram embeddings
model_skipgram = create_gru_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_matrix_skipgram.shape[1],
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout_rate=dropout_rate,
    max_len=max_len,
    embedding_matrix=embedding_matrix_skipgram
)

# Compile model
model_skipgram.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks_sg = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model_skipgram.h5', monitor='val_accuracy', 
                    save_best_only=True, mode='max')
]

print("\n Training GRU with Skip-gram embeddings...")
print(model_skipgram.summary())


 Training GRU with Skip-gram embeddings...


/Users/sola/.local/share/virtualenvs/text-classification-groupwork-ObVFzkD4/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_5 (Embedding)         │ (None, 100, 100)       │       641,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 100, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_10                │ (None, 100, 256)       │       176,640 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_11                │ (None, 256)            │       296,448 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,131,301 (4.32 MB)

 Trainable params: 489,601 (1.87 MB)

 Non-trainable params: 641,700 (2.45 MB)

None


In [ ]:
# Train model
history_skipgram = model_skipgram.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=15,
    batch_size=64,
    callbacks=callbacks_sg,
    verbose=1
)

Epoch 1/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.8504 - loss: 0.4072

53/53 ━━━━━━━━━━━━━━━━━━━━ 13s 178ms/step - accuracy: 0.9010 - loss: 0.2853 - val_accuracy: 0.9695 - val_loss: 0.0974
Epoch 2/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 183ms/step - accuracy: 0.9668 - loss: 0.1136 - val_accuracy: 0.9695 - val_loss: 0.0954
Epoch 3/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 13s 254ms/step - accuracy: 0.9704 - loss: 0.1028 - val_accuracy: 0.9686 - val_loss: 0.0929
Epoch 4/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 13s 239ms/step - accuracy: 0.9713 - loss: 0.0944 - val_accuracy: 0.9686 - val_loss: 0.1013
Epoch 5/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 13s 241ms/step - accuracy: 0.9704 - loss: 0.0905 - val_accuracy: 0.9686 - val_loss: 0.0948
Epoch 6/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 13s 249ms/step - accuracy: 0.9743 - loss: 0.0887 - val_accuracy: 0.9686 - val_loss: 0.1035
Epoch 7/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 257ms/step - accuracy: 0.9701 - loss: 0.0886 - val_accuracy: 0.9695 - val_loss: 0.0985
Epoch 8/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.9733 - loss: 0.0982

53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 265ms/step - accuracy: 0.9698 - loss: 0.0960 - val_accuracy: 0.9722 - val_loss: 0.0897
Epoch 9/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 272ms/step - accuracy: 0.9737 - loss: 0.0828 - val_accuracy: 0.9695 - val_loss: 0.0942
Epoch 10/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 268ms/step - accuracy: 0.9698 - loss: 0.0925 - val_accuracy: 0.9704 - val_loss: 0.0931
Epoch 11/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 16s 298ms/step - accuracy: 0.9743 - loss: 0.0851 - val_accuracy: 0.9659 - val_loss: 0.0940
Epoch 12/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 270ms/step - accuracy: 0.9725 - loss: 0.0825 - val_accuracy: 0.9704 - val_loss: 0.0922
Epoch 13/15
32/53 ━━━━━━━━━━━━━━━━━━━━ 5s 256ms/step - accuracy: 0.9788 - loss: 0.0754

### 8.3 Evaluate Skip-gram Model

In [ ]:
# Evaluate on test set
test_metrics_sg, preds_sg = evaluate_model(model_skipgram, X_test_seq, y_test)

print("\n Skip-gram Model - Test Results:")
print(f"Accuracy:  {test_metrics_sg['accuracy']:.4f}")
print(f"F1 Score:  {test_metrics_sg['f1']:.4f}")
print(f"Precision: {test_metrics_sg['precision']:.4f}")
print(f"Recall:    {test_metrics_sg['recall']:.4f}")

In [ ]:
# Save results
results_skipgram = {
    'embedding': 'Skip-gram',
    'test_metrics': test_metrics_sg,
    'train_history': history_skipgram.history
}

# Confusion matrix
cm_sg = confusion_matrix(y_test, preds_sg)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_sg, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
plt.title('Confusion Matrix - GRU with Skip-gram')
plt.savefig('../results/figures/gru_skipgram_confusion.png', dpi=300, bbox_inches='tight')
plt.show()

## EXPERIMENT 3: GRU with CBOW Embeddings

In [18]:
print("\n" + "=" * 60)
print("EXPERIMENT 3: GRU with CBOW Embeddings")
print("=" * 60)


EXPERIMENT 3: GRU with CBOW Embeddings


### 9.1 Train CBOW Embeddings

In [40]:
print("=" * 60)
print("EXPERIMENT 3: GRU with CBOW Embeddings - Training CBOW")
print("=" * 60)

# Get word_index from tokenizer
word_index = tokenizer_tfidf.word_index

try:
    from embeddings.embedding_cbow import CBOWEmbedding
    print("✓ Using existing CBOWEmbedding module")
    
    # Initialize CBOW
    cbow = CBOWEmbedding(vector_size=100, window=5, min_count=2)
    
    # Train on all text
    cbow.train(all_texts)
    
    # Get embedding matrix
    embedding_matrix_cbow = cbow.get_embedding_matrix(word_index)
    print(f"✓ CBOW embedding matrix shape: {embedding_matrix_cbow.shape}")
    
except ImportError:
    print("⚠️ CBOWEmbedding module not found, using Word2Vec")
    
    # Train Word2Vec CBOW model
    sentences = [text.split() for text in all_texts]

    cbow_model = Word2Vec(
        sentences=sentences,
        vector_size=100,
        window=5,
        min_count=2,
        sg=0,  # sg=0 for CBOW
        workers=4,
        seed=SEED
    )

    # Create embedding matrix
    embedding_matrix_cbow = np.zeros((vocab_size, embedding_dim))
    
    for word, idx in word_index.items():
        if idx < vocab_size:
            if word in cbow_model.wv:
                embedding_matrix_cbow[idx] = cbow_model.wv[word]
            else:
                # Initialize with small random values
                embedding_matrix_cbow[idx] = np.random.normal(scale=0.1, size=(embedding_dim,))
    
    print(f"✓ CBOW embedding matrix shape: {embedding_matrix_cbow.shape}")

EXPERIMENT 3: GRU with CBOW Embeddings - Training CBOW
✓ Using existing CBOWEmbedding module
Training CBOW Word2Vec embeddings...
CBOW training complete
Vocabulary size: 3985
Embedding dimension: 100
✓ CBOW embedding matrix shape: (6417, 100)


### 9.2 Train GRU with CBOW Embeddings

In [41]:
# Create model with CBOW embeddings
model_cbow = create_gru_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_matrix_cbow.shape[1],
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout_rate=dropout_rate,
    max_len=max_len,
    embedding_matrix=embedding_matrix_cbow
)

# Compile model
model_cbow.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks_cbow = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model_cbow.h5', monitor='val_accuracy', 
                    save_best_only=True, mode='max')
]

print("\n Training GRU with CBOW embeddings...")
print(model_cbow.summary())

# Train model
history_cbow = model_cbow.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=15,
    batch_size=64,
    callbacks=callbacks_cbow,
    verbose=1
)



 Training GRU with CBOW embeddings...


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, 100, 100)       │       641,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 100, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ (None, 100, 256)       │       176,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 256)            │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,131,301 (4.32 MB)

 Trainable params: 489,601 (1.87 MB)

...

None
Epoch 1/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.8482 - loss: 0.4559

53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 187ms/step - accuracy: 0.8615 - loss: 0.4252 - val_accuracy: 0.8655 - val_loss: 0.4047
Epoch 2/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 186ms/step - accuracy: 0.8659 - loss: 0.3914 - val_accuracy: 0.8655 - val_loss: 0.3247
Epoch 3/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 256ms/step - accuracy: 0.8659 - loss: 0.3293 - val_accuracy: 0.8655 - val_loss: 0.3225
Epoch 4/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 258ms/step - accuracy: 0.8674 - loss: 0.3221 - val_accuracy: 0.8655 - val_loss: 0.3165
Epoch 5/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 13s 253ms/step - accuracy: 0.8656 - loss: 0.3244 - val_accuracy: 0.8655 - val_loss: 0.3193
Epoch 6/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 266ms/step - accuracy: 0.8659 - loss: 0.3176 - val_accuracy: 0.8655 - val_loss: 0.3253
Epoch 7/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 260ms/step - accuracy: 0.8659 - loss: 0.3068 - val_accuracy: 0.8655 - val_loss: 0.3085
Epoch 8/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 270ms/step - accuracy: 0.8659 - loss: 0.3067 - val_accuracy: 0.865

53/53 ━━━━━━━━━━━━━━━━━━━━ 15s 279ms/step - accuracy: 0.8668 - loss: 0.2771 - val_accuracy: 0.8717 - val_loss: 0.2847
Epoch 14/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.8750 - loss: 0.2741

53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 267ms/step - accuracy: 0.8728 - loss: 0.2769 - val_accuracy: 0.9274 - val_loss: 0.2449
Epoch 15/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - accuracy: 0.8875 - loss: 0.2647

53/53 ━━━━━━━━━━━━━━━━━━━━ 13s 255ms/step - accuracy: 0.8833 - loss: 0.2750 - val_accuracy: 0.9336 - val_loss: 0.2313


### 9.3 Evaluate CBOW Model

In [53]:


# Evaluate on test set
test_metrics_cbow, preds_cbow = evaluate_model(model_cbow, X_test_seq, y_test)

print("\n📊 CBOW Model - Test Results:")
print(f"Accuracy:  {test_metrics_cbow['accuracy']:.4f}")
print(f"F1 Score:  {test_metrics_cbow['f1']:.4f}")
print(f"Precision: {test_metrics_cbow['precision']:.4f}")
print(f"Recall:    {test_metrics_cbow['recall']:.4f}")

# Save results
results_cbow = {
    'embedding': 'CBOW',
    'test_metrics': test_metrics_cbow,
    'train_history': history_cbow.history
}

# Confusion matrix
cm_cbow = confusion_matrix(y_test, preds_cbow)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_cbow, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
plt.title('Confusion Matrix - GRU with CBOW')
plt.savefig('../results/figures/gru_cbow_confusion.png', dpi=300, bbox_inches='tight')
plt.show()


📊 CBOW Model - Test Results:
Accuracy:  0.9417
F1 Score:  0.7303
Precision: 0.9565
Recall:    0.5906


ModuleNotFoundError: No module named 'matplotlib_inline'

## 10. Comparison of All Embeddings

In [32]:
print("\n" + "=" * 60)
print("COMPARISON OF ALL EMBEDDINGS")
print("=" * 60)

# Create comparison table
comparison_data = []
for results in [results_tfidf, results_skipgram, results_cbow]:
    row = {
        'Embedding': results['embedding'],
        'Accuracy': results['test_metrics']['accuracy'],
        'F1 Score': results['test_metrics']['f1'],
        'Precision': results['test_metrics']['precision'],
        'Recall': results['test_metrics']['recall']
    }
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)
print("\n📈 Performance Comparison:")
print(comparison_df.to_string(index=False))

# Save comparison
comparison_df.to_csv('../results/tables/gru_comparison.csv', index=False)
print("\n Comparison saved to: ../results/tables/gru_comparison.csv")

# Visualize comparison
plt.figure(figsize=(10, 6))
x = np.arange(len(comparison_df))
width = 0.2

plt.bar(x - width, comparison_df['Accuracy'], width, label='Accuracy', alpha=0.8)
plt.bar(x, comparison_df['F1 Score'], width, label='F1 Score', alpha=0.8)
plt.bar(x + width, comparison_df['Precision'], width, label='Precision', alpha=0.8)

plt.xlabel('Embedding Type')
plt.ylabel('Score')
plt.title('GRU Performance with Different Embeddings')
plt.xticks(x, comparison_df['Embedding'])
plt.legend()
plt.ylim(0.8, 1.0)
plt.grid(True, alpha=0.3)

# Add value labels
for i, (acc, f1, prec) in enumerate(zip(comparison_df['Accuracy'], 
                                        comparison_df['F1 Score'], 
                                        comparison_df['Precision'])):
    plt.text(i - width, acc + 0.01, f'{acc:.3f}', ha='center', va='bottom', fontsize=9)
    plt.text(i, f1 + 0.01, f'{f1:.3f}', ha='center', va='bottom', fontsize=9)
    plt.text(i + width, prec + 0.01, f'{prec:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('../results/figures/gru_embedding_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


COMPARISON OF ALL EMBEDDINGS


NameError: name 'results_skipgram' is not defined

## 11. Training Curves Comparison

In [33]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Loss curves
embeddings = ['TF-IDF', 'Skip-gram', 'CBOW']
histories = [history_tfidf, history_skipgram, history_cbow]

for i, (embedding, history) in enumerate(zip(embeddings, histories)):
    axes[0, i].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0, i].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, i].set_title(f'{embedding} - Loss')
    axes[0, i].set_xlabel('Epoch')
    axes[0, i].set_ylabel('Loss')
    axes[0, i].legend()
    axes[0, i].grid(True, alpha=0.3)

# Accuracy curves
for i, (embedding, history) in enumerate(zip(embeddings, histories)):
    axes[1, i].plot(history.history['accuracy'], label='Train Acc', linewidth=2)
    axes[1, i].plot(history.history['val_accuracy'], label='Val Acc', linewidth=2)
    axes[1, i].set_title(f'{embedding} - Accuracy')
    axes[1, i].set_xlabel('Epoch')
    axes[1, i].set_ylabel('Accuracy')
    axes[1, i].legend()
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/gru_training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

ModuleNotFoundError: No module named 'matplotlib_inline'

## 12. Error Analysis

In [ ]:
print("\n" + "=" * 60)
print("ERROR ANALYSIS")
print("=" * 60)

# Collect misclassified samples
errors = []
for i, (text, true_label, pred_tfidf, pred_sg, pred_cbow) in enumerate(zip(
    X_test[:100],  # First 100 test samples
    y_test[:100],
    preds_tfidf[:100],
    preds_sg[:100],
    preds_cbow[:100]
)):
    if not (true_label == pred_tfidf == pred_sg == pred_cbow):
        errors.append({
            'text': text[:50] + '...' if len(text) > 50 else text,
            'true': 'spam' if true_label == 1 else 'ham',
            'tfidf_pred': 'spam' if pred_tfidf == 1 else 'ham',
            'skipgram_pred': 'spam' if pred_sg == 1 else 'ham',
            'cbow_pred': 'spam' if pred_cbow == 1 else 'ham',
            'all_agree': (pred_tfidf == pred_sg == pred_cbow)
        })

errors_df = pd.DataFrame(errors)
print(f"\nFound {len(errors)} misclassified samples in first 100 test examples")

if len(errors) > 0:
    print("\n🔍 Sample Misclassifications:")
    print(errors_df.head().to_string(index=False))
    
    # Save error analysis
    errors_df.to_csv('../results/tables/gru_error_analysis.csv', index=False)
    print("\n Error analysis saved to: ../results/tables/gru_error_analysis.csv")

## 13. Summary and Conclusion

In [ ]:


print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)

# Find best performing model
best_model = max([results_tfidf, results_skipgram, results_cbow], 
                 key=lambda x: x['test_metrics']['f1'])

print(f"\n Best Performing Embedding: {best_model['embedding']}")
print(f"   F1 Score: {best_model['test_metrics']['f1']:.4f}")
print(f"   Accuracy: {best_model['test_metrics']['accuracy']:.4f}")

print("\n Key Observations:")
print("1. All embeddings achieved high performance (>0.95 F1)")
print("2. [Your observation about which embedding worked best]")
print("3. [Any patterns in misclassifications]")
print("4. [Training time/convergence observations]")

print("\n Files Generated:")
print("  Results:")
print("    ../results/tables/gru_comparison.csv")
print("    ../results/tables/gru_error_analysis.csv")
print("  Figures:")
print("    ../results/figures/gru_tfidf_confusion.png")
print("    ../results/figures/gru_skipgram_confusion.png")
print("    ../results/figures/gru_cbow_confusion.png")
print("    ../results/figures/gru_embedding_comparison.png")
print("    ../results/figures/gru_training_curves.png")

print("\n GRU EXPERIMENTS COMPLETE!")

ImportError: cannot import name 'TextCleaner' from 'preprocessing.clean_text' (/Users/sola/text-classification-groupwork/notebooks/../preprocessing/clean_text.py)